[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/koya-jp/AA-google-colab-kohya/blob/master/webui_sdxl_run_v1.6.0.ipynb)

#### ***初期設定***

In [ ]:
# 環境の準備　（ターミナルで実行）

# 新しい環境を作る
conda create -y --name sd_env python=3.10.6
conda activate sd_env
conda install -y ipykernel

# 環境をリセット（削除）
conda activate sd_env
conda deactivate
conda remove -y -n sd_env --all

In [ ]:
# 環境構築
%cd /home/studio-lab-user/
!python -V

# 必要なパッケージをインストール
%conda install -y aria2
!pip install torchvision==0.15.2 -U
!pip install xformers==0.0.20 -U
!pip install triton==2.0.0 -U
%conda install -y -c conda-forge libglib

# stable diffusion webuiのリポジトリをクローン （コミットハッシュ → 5ef669de080814067961f28357256e8fe27544f4）
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /home/studio-lab-user/ui
!df -h

#### ***Model, VAE***

In [ ]:
lode_mode="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"
# !rm -rf /home/studio-lab-user/.cache # キャッシュの削除

# Model
model_dir="/home/studio-lab-user/ui/models/Stable-diffusion"
%cd {model_dir}
!rm -rf {model_dir}/*
# !rm -rf {model_dir}/MeinaMix.safetensors #削除したいファイルを入力

# SDXL
!{lode_mode} https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors -d {model_dir} -o sd_xl_base_1.0_0.9vae.safetensors
# !{lode_mode} https://civitai.com/api/download/models/145893 -d {model_dir} -o BreakDomainXL.safetensors

# Refiner
!{lode_mode} https://huggingface.co/ckpt/sd_xl_refiner_1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors -d {model_dir} -o sd_xl_refiner_1.0.safetensors

# SD2.0以前
# !{lode_mode} https://civitai.com/api/download/models/119057 -d {model_dir} -o MeinaMix.safetensors
# !{lode_mode} https://civitai.com/api/download/models/113479 -d {model_dir} -o beautifulRealistic_v60.safetensors

# VAE
vae_dir="/home/studio-lab-user/ui/models/VAE"
!rm {vae_dir}/*
!{lode_mode} https://huggingface.co/ckpt/sdxl_vae/resolve/main/sdxl_vae.safetensors -d {vae_dir} -o sdxl_vae.vae.safetensors
# !{lode_mode} https://huggingface.co/okozk/custom_models/resolve/main/vae_mse_840000_ema_semi.pt -d {vae_dir} -o vae_mse_840000_ema_semi.vae.pt

# 確認
!ls -la {model_dir}
!ls -la {vae_dir}
!df -h

#### ***embedings***

In [ ]:
lode_mode="aria2c --console-log-level=error -c -q -x 16 -s 16 -k 1M"
# embedings
base_dir="/home/studio-lab-user/ui/embeddings"
%cd {base_dir}
!rm -rf {base_dir}/*

!{lode_mode} https://civitai.com/api/download/models/134424 -d {base_dir} -o unaestheticX-NegL.safetensors # Flat2 のように使う
!{lode_mode} https://civitai.com/api/download/models/134583 -d {base_dir} -o negativeXL_D.safetensors # 露出度のネガティブ 0.5 ~ 0.7 くらいで普通。
!{lode_mode} https://civitai.com/api/download/models/139645 -d {base_dir} -o peopleneg.safetensors # 人間のネガティブな特徴を学習。実写用
!{lode_mode} https://civitai.com/api/download/models/128882 -d {base_dir} -o SimplepositiveXLv1.safetensors # Simpleなポジティブ要素

# 確認
!ls -la {base_dir}
!df -h

#### ***Lora***

In [ ]:
lode_mode="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"
base_dir="/home/studio-lab-user/ui/models/Lora"
!rm -rf {base_dir}/* #リセット

# 容量を逼迫するので利用するものだけにする。
# !{lode_mode} https://civitai.com/api/download/models/121538 -d {base_dir} -o watercolor-style.safetensors
# !{lode_mode} https://civitai.com/api/download/models/136749 -d {base_dir} -o ChibiStyleXL.safetensors
# !{lode_mode} https://civitai.com/api/download/models/138938 -d {base_dir} -o Linear-Manga-Style.safetensors
# !{lode_mode} https://civitai.com/api/download/models/131266 -d {base_dir} -o JOJO-Style.safetensors
!{lode_mode} https://civitai.com/api/download/models/137124 -d {base_dir} -o DreamART-Style.safetensors
# !{lode_mode} https://civitai.com/api/download/models/126807 -d {base_dir} -o Photorealistic-Slider.safetensors
# !{lode_mode} https://civitai.com/api/download/models/127510 -d {base_dir} -o greg_rutkowski_xl_2.safetensors
# !{lode_mode} https://civitai.com/api/download/models/118427 -d {base_dir} -o perfect-eyes-xl.safetensors
!{lode_mode} https://civitai.com/api/download/models/62833 -d {base_dir} -o detail-tweaker-xl.safetensors

# 確認
!ls -la {base_dir}
!df -h

#### ***拡張機能***

In [ ]:
# 拡張機能の追加
base_dir="/home/studio-lab-user/ui/extensions"
lode_mode="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"
%cd {base_dir}
# !rm -rf OO #削除したいファイルを入力
!rm -rf {base_dir}/*

!git clone https://github.com/camenduru/sd-civitai-browser {base_dir}/civitai-browser
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser {base_dir}/images-browser
!git clone https://github.com/zanllp/sd-webui-infinite-image-browsing {base_dir}/infinite-image-browsing
!git clone https://github.com/Mikubill/sd-webui-controlnet {base_dir}/controlnet
!git clone https://github.com/anonCantCode/sd-webui-send-to-controlnet {base_dir}/send-to-controlnet
!git clone https://github.com/jexom/sd-webui-depth-lib {base_dir}/depth-lib
!git clone https://github.com/richrobber2/canvas-zoom {base_dir}/canvas-zoom
# !git clone https://github.com/huchenlei/sd-webui-openpose-editor {base_dir}/openpose-editor
# !git clone https://github.com/fkunn1326/openpose-editor {base_dir}/openpose-editor
# !git clone https://github.com/hnmr293/posex {base_dir}/posex
# !git clone https://github.com/nonnonstop/sd-webui-3d-open-pose-editor {base_dir}/3d-open-pose-editor
# !git clone https://github.com/camenduru/tunnels {base_dir}/tunnels
# !git clone https://github.com/etherealxx/batchlinks-webui {base_dir}/batchlinks-webui
# !git clone https://github.com/catppuccin/stable-diffusion-webui {base_dir}/catppuccin
!git clone https://github.com/Haoming02/sd-webui-prompt-format {base_dir}/prompt-format
# !git clone https://github.com/eideehi/sd-webui-better-prompt {base_dir}/better-prompt
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete {base_dir}/tagcomplete
!git clone https://github.com/chrisgoringe/Styles-Editor {base_dir}/Styles-Editor
!git clone https://github.com/opparco/stable-diffusion-webui-two-shot {base_dir}/two-shot
# !git clone https://github.com/bbc-mc/sdweb-clip-changer {base_dir}/clip-changer
!git clone https://github.com/hnmr293/sd-webui-cutoff {base_dir}/cutoff
# !git clone https://github.com/toriato/stable-diffusion-webui-wd14-tagger {base_dir}/wd14-tagger
!git clone https://github.com/Bing-su/adetailer {base_dir}/adetailer
!git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 {base_dir}/multidiffusion-upscaler
!git clone https://github.com/alemelis/sd-webui-ar {base_dir}/sd-webui-ar
!git clone https://github.com/Zyin055/Config-Presets {base_dir}/Config-Presets
!git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-haku-img  {base_dir}/haku-img
!git clone https://github.com/yankooliveira/sd-webui-photopea-embed  {base_dir}/photopea-embed
!git clone https://github.com/arenasys/stable-diffusion-webui-model-toolkit {base_dir}/model-toolkit
!git clone https://github.com/aka7774/sd_katanuki {base_dir}/sd_katanuki
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg {base_dir}/rembg

# リセット
!git reset --hard
!git -C /home/studio-lab-user/ui/repositories/stable-diffusion-stability-ai reset --hard
# %cd {base_dir}/prompt-all-in-one
# !git checkout a46e7f24a9aa4817a6fe81647aa2c85be096f72b

# sd-webui-ar の拡張機能に SDXL の推奨サイズを書き込み
f = open("/home/studio-lab-user/ui/extensions/sd-webui-ar/resolutions.txt", "w")
text = """1, 1024, 1024 # 1:1 Square
2, 886, 1182 # 3:4 Photo
3, 836, 1254 # 2:3 Landscape
4, 768, 1344 # 9:16 Vertical
1/2, 512, 512 # 1:1 Square
2/4, 443, 591 # 3:4 Photo
3/6, 418, 627 # 2:3 Landscape
4/8, 384, 672 # 9:16 Vertical"""
f.write(text)
f.close()

# 確認
%ls -la {base_dir}
!df -h

#### ***ControlNet Models***

In [ ]:
# ControlNetのモデルの追加
lode_mode="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"
base_dir="/home/studio-lab-user/ui/models/ControlNet"

!rm -rf {base_dir}/*

# SDXL対応版
!{lode_mode} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-canny-rank256.safetensors -d {base_dir} -o control-lora-canny-rank256.safetensors
!{lode_mode} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-depth-rank256.safetensors -d {base_dir} -o control-lora-depth-rank256.safetensors
# !{lode_mode} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-recolor-rank256.safetensors -d {base_dir} -o control-lora-recolor-rank256.safetensors
# !{lode_mode} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-sketch-rank256.safetensors -d {base_dir} -o control-lora-sketch-rank256.safetensors

# 軽量版
# !{lode_mode} https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d {base_dir} -o control_v11p_sd15_canny_fp16.safetensors
# !{lode_mode} https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d {base_dir} -o control_v11f1p_sd15_depth_fp16.safetensors
# !{lode_mode} https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d {base_dir} -o control_v11p_sd15_inpaint_fp16.safetensors

# Full Version
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d {base_dir} -o control_v11f1p_sd15_depth_fp16.safetensors
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d {base_dir} -o control_v11p_sd15_inpaint_fp16.safetensors
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d {base_dir} -o control_v11p_sd15_canny_fp16.safetensors
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d {lode_mode} -o control_v11f1e_sd15_tile_fp16.safetensors
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d {lode_mode} -o control_v11e_sd15_ip2p_fp16.safetensors
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d {lode_mode} -o control_v11e_sd15_shuffle_fp16.safetensors
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d {lode_mode} -o control_v11p_sd15_lineart_fp16.safetensors
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d {lode_mode} -o control_v11p_sd15_mlsd_fp16.safetensors
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d {lode_mode} -o control_v11p_sd15_normalbae_fp16.safetensors
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d {lode_mode} -o control_v11p_sd15_openpose_fp16.safetensors
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d {lode_mode} -o control_v11p_sd15_scribble_fp16.safetensors
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d {lode_mode} -o control_v11p_sd15_seg_fp16.safetensors
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d {lode_mode} -o control_v11p_sd15_softedge_fp16.safetensors
# !{lode_mode} https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d {lode_mode} -o control_v11p_sd15s2_lineart_anime_fp16.safetensors

# 確認
%cd {base_dir}
!ls -la {base_dir}
!df -h

#### ***depth-lib の画像を配置***

In [ ]:
# maps.zip をホームに用意しておくためにダウンロード
lode_mode="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"
maps_dir = "/home/studio-lab-user/ui/extensions/depth-lib/maps"
home = "/home/studio-lab-user"
!{lode_mode} https://huggingface.co/okozk/depth_maps/resolve/main/maps.zip -d {home} -o maps.zip

# 確認
!ls {maps_dir}

In [ ]:
# depth-lib の画像をの画像を準備する (1GB)
maps_dir = "/home/studio-lab-user/ui/extensions/depth-lib/maps"
delete_folder = "/home/studio-lab-user/Delete"

# 今日日付を取得
import datetime
today = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Deleteフォルダが無ければ作成
import os
if not os.path.exists(delete_folder):
    os.makedirs(delete_folder)

# Zipファイルを解答して不要なフォルダを削除する。
!unzip -d {maps_dir}/ /home/studio-lab-user/maps.zip > /dev/null
!mv /home/studio-lab-user/maps.zip {delete_folder}/maps_{today}.zip
!rm -rf {maps_dir}/__MACOSX

# 確認
!ls {maps_dir}

#### ***sd_katanuki***

In [ ]:
# sd_katanuki のフォルダを用意 ⭐ train-images.zip を用意しておく
!mkdir /home/studio-lab-user/sd_katanuki_in
!mkdir /home/studio-lab-user/sd_katanuki_out
!unzip -d /home/studio-lab-user/sd_katanuki_in /home/studio-lab-user/katanuki.zip > /dev/null
!rm -rf /home/studio-lab-user/sd_katanuki_in/__MACOSX

# 確認
!ls /home/studio-lab-user/sd_katanuki_in # | wc -l

In [ ]:
# sd_katanuki の結果をZipにする
%cd /home/studio-lab-user

# 今日日付を取得
import datetime
today = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Deleteフォルダが無ければ作成
delete_folder = "/home/studio-lab-user/Delete"
if not os.path.exists(delete_folder):
    os.makedirs(delete_folder)

# 作成したファイルを移動してZipする。
!tar -czf /home/studio-lab-user/sd_katanuki_out_{today}.tar.gz sd_katanuki_out
!mv /home/studio-lab-user/sd_katanuki_in /home/studio-lab-user/Delete/sd_katanuki_in_{today}
!mv /home/studio-lab-user/sd_katanuki_out /home/studio-lab-user/Delete/sd_katanuki_out_{today}
!mv /home/studio-lab-user/katanuki.zip /home/studio-lab-user/Delete/katanuki_{today}.zip

#### ***画像生成 の結果をZipにする***

In [ ]:
# 画像生成 の結果をZipにする。100件以上のフォルダは、100件ごとにZipにする。

import os
import glob
import datetime
import zipfile
import shutil

# 今日日付を取得
today = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# 圧縮したい画像のパス
# 画像のパス
img_paths = [f"/home/studio-lab-user/ui/outputs/{folder}" for folder in ["txt2img-images", "txt2img-grids", "img2img-images", "img2img-grids", "extras-images"]]

# ZIPファイルのパス
zip_paths = [f"/home/studio-lab-user/outputs_{folder}_{today}.zip" for folder in ["txt2img-images", "txt2img-grids", "img2img-images", "img2img-grids", "extras-images"]]

# 移動対象のフォルダのパス
move_paths = [f"{img_path}/*" for img_path in img_paths]

# 不要なフォルダのパス
delete_paths = [f"/home/studio-lab-user/Delete/{folder}/{today}" for folder in ["txt2img-images", "txt2img-grids", "img2img-images", "img2img-grids", "extras-images"]]


# ZIPファイルに圧縮する関数
def zip_files(img_path, zip_path):
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        # サブフォルダとファイルの全てのパスを取得
        for root, dirs, files in os.walk(img_path):
            for file in files:
                # ZIPファイルに書き込むパスを相対パスにする
                rel_path = os.path.relpath(os.path.join(root, file), img_path)
                # ZIPファイルに書き込む
                zf.write(os.path.join(root, file), rel_path)


# img_pathのファイル数が50以上あれば、1, 2のような名前でフォルダを作って、100件ずつ移動させる関数
def split_files(img_path):
    # img_pathのファイル数を取得
    file_count = len(os.listdir(img_path))

    # ファイル数が50以上ならば
    if file_count > 50:
        # フォルダの番号を初期化
        folder_num = 1
        # ファイルのリストを取得
        file_list = os.listdir(img_path)
        # ファイルのリストを100件ずつに分割
        file_chunks = [file_list[i:i+50] for i in range(0, file_count, 50)]
        # 分割したファイルのリストごとに処理
        for file_chunk in file_chunks:
            # フォルダの名前を作成
            folder_name = str(folder_num)
            # フォルダのパスを作成
            folder_path = os.path.join(img_path, "zip_"+folder_name)
            # フォルダがなければ作成
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            # フォルダにファイルを移動
            for file in file_chunk:
                file_path = os.path.join(img_path, file)
                shutil.move(file_path, folder_path)
            # フォルダの番号を更新
            folder_num += 1


# 画像のパスとZIPファイルのパスのペアごとに圧縮と移動を行う関数
def zip_and_move(img_path, zip_path, move_path, delete_path):
    # 画像のパスに何もフォルダやファイルがなければスキップする
    if len(os.listdir(img_path)) == 0:
        return

    # img_pathのファイル数が100以上あれば、1, 2のような名前でフォルダを作って、100件ずつ移動させる関数を呼び出す
    split_files(img_path)

    # img_pathのサブフォルダのリストを取得
    subfolder_list = os.listdir(img_path)

    # サブフォルダのリストごとに処理
    for subfolder in subfolder_list:
        # サブフォルダのパスを作成
        subfolder_path = os.path.join(img_path, subfolder)
        # Zipファイルの名前を作成（末尾にサブフォルダの名前を付け足す）
        zip_name = zip_path[:-4] + "_" + subfolder + ".zip"
        # Zipファイルに圧縮する関数を呼び出す
        zip_files(subfolder_path, zip_name)

    # 移動先のディレクトリがなければ作成する
    if not os.path.exists(delete_path):
        os.makedirs(delete_path)

    # ワイルドカードにマッチするパスのリストを取得
    move_list = glob.glob(move_path)

    # 移動を実行
    for file in move_list:
        shutil.move(file, delete_path)

    print(img_path)
    !ls {img_path}


# 容量の整理
!rm -rf /home/studio-lab-user/.cache # キャッシュの削除

# 画像のパスとZIPファイルのパスのペアごとに圧縮と移動を行う
for img_path, zip_path, move_path, delete_path in zip(img_paths, zip_paths, move_paths, delete_paths):
    zip_and_move(img_path, zip_path, move_path, delete_path)

# 予備
# today = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
# zip_path = f"/home/studio-lab-user/outputs-{today}.zip"
# img_path = "Delete/txt2img-images/20230816_130141/2023-08-16/2"
# zip_files("/home/studio-lab-user/"+img_path,zip_path)

#### ***⭐ 実行 ----------------------------------------------------------------------------***

In [ ]:
# 現在の空き容量を表示
# !rm -rf /home/studio-lab-user/.cache # キャッシュの削除
!df -h

# Web ui 起動
%cd /home/studio-lab-user/ui
ngrok_token="YOUR NGROK TOKEN"
# home="/home/studio-lab-user/ui"
# %cd {home}
# !sed -i -e '''/from modules import launch_utils/a\import os''' {home}/launch.py
# !sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {home}/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {home}/launch.py
# !sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' {home}/modules/shared.py
ARGS = "\"--listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --no-half-vae --ngrok " + ngrok_token + "\""
!COMMANDLINE_ARGS=$ARGS python launch.py

#### ***容量の整理***

In [ ]:
# 容量の確認
!df -h
!free -h

In [ ]:
# 容量の整理
!rm -rf /home/studio-lab-user/.ipynb_checkpoints
!rm -rf /home/studio-lab-user/.cache # キャッシュの削除
# !rm -rf /home/studio-lab-user/.conda #リセット（環境を作りなおすなら実行）
!ls -la /home/studio-lab-user/

In [ ]:
# model-toolkitで軽量化したモデルを Hugging face にアップロードする
Model_Directory="/home/studio-lab-user/ui/models/VAE" # 対象のディレクトリ
File_Name="vae-ft-mse-840000-ema-pruned.vae-0869.vae.pt" # 対象のファイル

HF_File_Name = "vae_mse_840000_ema_semi.pt" # Hugging face のファイル名
User_Repository ="Hugging face のリポジトリ名"  # Hugging face のリポジトリ名

Upload_File = Model_Directory + "/" + File_Name
Token = "Hugging face トークン" #Hugging face トークン

%cd {Model_Directory}

# アップロード
from huggingface_hub import upload_file
upload_file(path_or_fileobj=Upload_File, path_in_repo=HF_File_Name, repo_id=User_Repository, token=Token)